In [2]:
import numpy as np
import pandas as pd
# np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)
# please show all columns
pd.set_option("display.max_columns", 60)

In [3]:
!pwd

/c/Users/saus/Documents/repos/springboard/solar-cap-1/wrangle


In [7]:
! ls -al ../local/data/LBNL_openpv_tts_data/

total 1773893
-rw-r--r-- 1 saus 197609       706 Jan 28 16:42 #README.txt#
drwxr-xr-x 1 saus 197609         0 Jan 28 16:42 .
-rw-r--r-- 1 saus 197609        27 Jan 28 16:42 .#README.txt
drwxr-xr-x 1 saus 197609         0 Jan 19 21:24 ..
drwxr-xr-x 1 saus 197609         0 Jan 19 21:31 .ipynb_checkpoints
-rw-r--r-- 1 saus 197609    859629 Nov  9 21:36 PublicDataFile_UserGuide.pdf
-rw-r--r-- 1 saus 197609       699 Nov 13 22:56 README.txt
-rw-r--r-- 1 saus 197609 501984387 Jan 17 22:32 TTS.csv
-rw-r--r-- 1 saus 197609   3977507 Jan 15 15:35 TTS10k.csv
-rw-r--r-- 1 saus 197609 185901845 Nov  9 21:36 TTSX_LBNL_OpenPV_public_file_p1.xlsx
-rw-r--r-- 1 saus 197609  95456887 Nov  9 21:36 TTSX_LBNL_OpenPV_public_file_p2.xlsx
-rw-r--r-- 1 saus 197609 226627007 Jan 19 18:18 live20180119.csv
-rw-r--r-- 1 saus 197609 232427884 Jan 18 16:01 live_20180118
-rw-r--r-- 1 saus 197609 190295601 Jan 19 22:34 ttsclean20180119.csv
-rw-r--r-- 1 saus 197609 187478829 Jan 23 21:33 ttsclean20180123.csv
-rw-r--r--

In [6]:
# where are the reads?
! grep read_csv *tts*.ipynb

tts_2nd_look.ipynb:    "dfmini = pd.read_csv('../local/data/LBNL_openpv_tts_data/TTS.csv',\n",
tts_5.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/live_20180118',\n",
tts_6.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/live_20180118',\n",
tts_6.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/live20180119.csv', index_col='row_id',\n",
tts_7.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180119.csv',\n",
tts_8.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180119.csv',\n",
tts_9.ipynb:    "dftts = pd.read_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180119.csv',\n",
tts_scratch.ipynb:      "d_q_and_c_1.ipynb:    \"dfFull = pd.read_csv('../local/data/live_20171219.csv')\"\n",
tts_scratch.ipynb:      "d_q_and_c_2.ipynb:    \"dfFull = pd.read_csv('../local/data/live_20171219_1.csv', index_col='date_installed', parse_dates=True)\"\n",
tts_scratch.ipynb:      "d_q_and_

In [8]:
! grep encoding *tts*.ipynb

tts_5.ipynb:    "                    encoding = 'utf-8',\n",
tts_6.ipynb:    "                    encoding = 'utf-8',\n",
tts_6.ipynb:    "dftts.to_csv('../local/data/LBNL_openpv_tts_data/live20180119.csv', encoding = 'utf-8')"
tts_6.ipynb:    "                   encoding = 'utf-8', dtype={'zipcode' : np.object}) "
tts_6.ipynb:    "ttsClean.to_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180119.csv', encoding='utf-8')"
tts_7.ipynb:    "                    encoding = 'utf-8', \n",
tts_8.ipynb:    "                    encoding = 'utf-8', \n",
tts_9.ipynb:    "                    encoding = 'utf-8', \n",
tts_9.ipynb:    "dfLive.to_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180123.csv', encoding='utf-8')"
tts_9.ipynb:    "dfLive.to_csv('../local/data/LBNL_openpv_tts_data/ttsclean20180123.csv', encoding='utf-8')"


In [9]:
# These variableS become boolean:
# 'Appraised Value Flag' comes in right already so doesn't need converter
# array(['New Construction', 'Tracking', 'Ground Mounted', 
#        'Battery System', 'Third-Party Owned', 'Self-Installed', 'BIPV Module #1', 
#        'BIPV Module #2', 'BIPV Module #3', 'Microinverter', 'DC Optimizer'], dtype=object)

# change to take string arg rather than int...
def fnA(val):
    ''' take string value of "0", "1", "-9999" and turn into True, False and NaN'''
    if val in (['0', '1', '-9999']):
        return {'0' : False,
                '1' : True,
                '-9999' : np.nan}[val]
    else:
        raise TypeError('bad arg to fnA: {}, type={}'.format(val, type(val))) 

# print((fnA('0'), fnA('1'), fnA('-9999')))
# fnA('hugely successful')


myConverters = {'New Construction' : fnA, 
                'Tracking'         : fnA, 
                'Ground Mounted'   : fnA, 
                'Battery System'   : fnA, 
                'Third-Party Owned': fnA, 
                'Self-Installed'   : fnA, 
                'BIPV Module #1'   : fnA, 
                'BIPV Module #2'   : fnA, 
                'BIPV Module #3'   : fnA, 
                'Microinverter'    : fnA, 
                'DC Optimizer'     : fnA
               }


In [10]:
dfmini = pd.read_csv('../local/data/LBNL_openpv_tts_data/TTS.csv',
                     index_col='row_id',
                     parse_dates=['Installation Date'],
                     converters=myConverters,
                     # converters=None, 
                     # low_memory=False,
                     # dtype={'Third-Party Owned' : np.bool},
                     na_values=[-9999],
                     nrows = 100000
                    ); dfmini.head()

C:\Users\saus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (16,17,18,19,20,23,26,28,42,43,50,51,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,index,Data Provider,System ID (from Data Provider),System ID (Tracking the Sun),Installation Date,System Size,Total Installed Price,Appraised Value Flag,Sales Tax Cost,Rebate or Grant,Performance-Based Incentive (Annual Payment),Performance-Based Incentives (Duration),Feed-in Tariff (Annual Payment),Feed-in Tariff (Duration),Customer Segment,New Construction,Tracking,Tracking Type,Ground Mounted,Battery System,Zip Code,City,County,State,Utility Service Territory,Third-Party Owned,Installer Name,Self-Installed,Azimuth #1,Azimuth #2,Azimuth #3,Tilt #1,Tilt #2,Tilt #3,Module Manufacturer #1,Module Manufacturer #2,Module Manufacturer #3,Module Model #1,Module Model #2,Module Model #3,Module Technology #1,Module Technology #2,Module Technology #3,BIPV Module #1,BIPV Module #2,BIPV Module #3,Module Efficiency #1,Module Efficiency #2,Module Efficiency #3,Inverter Manufacturer,Inverter Model,Microinverter,DC Optimizer
row_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0,Arkansas State Energy Office,NaN,AR_EDC_1,2010-04-29,2.016,14558.0,False,510.762764,0.0,3644.64,1,0.0,0,RES,NaN,False,Fixed,False,NaN,71953.0,Mena,Polk,AR,SWEPCO,False,Liberty Solar Solutions,False,NaN,NaN,NaN,NaN,NaN,NaN,Sharp,Sharp,Sharp,ND-224UC1,no match,no match,Multi-c-Si,NaN,NaN,False,False,False,0.142431,NaN,NaN,Enphase Energy,NaN,True,False
1,1,Arkansas State Energy Office,NaN,AR_EDC_2,2010-04-26,3.360,26096.0,False,851.271273,0.0,7210.50,1,0.0,0,RES,NaN,False,Fixed,False,NaN,72641.0,Jasper,Newton,AR,Carroll Electric,False,Liberty Solar Solutions,False,NaN,NaN,NaN,NaN,NaN,NaN,Sharp,Sharp,Sharp,ND-224UC1,no match,no match,Multi-c-Si,NaN,NaN,False,False,False,0.142431,NaN,NaN,Enphase Energy,NaN,True,False
2,2,Arkansas State Energy Office,NaN,AR_EDC_3,2010-04-20,13.440,91139.0,False,3405.085091,0.0,25178.97,1,0.0,0,RES,NaN,False,Fixed,False,NaN,71801.0,Hope,Hempstead,AR,Hope Water & Light,False,Liberty Solar Solutions,False,NaN,NaN,NaN,NaN,NaN,NaN,Sharp,Sharp,Sharp,ND-224UC1,no match,no match,Multi-c-Si,NaN,NaN,False,False,False,0.142431,NaN,NaN,Enphase Energy,NaN,True,False
3,3,Arkansas State Energy Office,NaN,AR_EDC_4,2010-04-21,5.520,40043.0,False,1398.517091,0.0,10724.34,1,0.0,0,RES,NaN,False,Fixed,False,NaN,71909.0,Hot Springs Village,Saline,AR,First Electric,False,Liberty Solar Solutions,False,NaN,NaN,NaN,NaN,NaN,NaN,Sharp,Sharp,Sharp,NU-U230F3,no match,no match,Mono-c-Si,NaN,NaN,False,False,False,0.141090,NaN,NaN,Enphase Energy,NaN,True,False
4,4,Arkansas State Energy Office,NaN,AR_EDC_5,2010-04-22,2.530,21497.0,False,640.987000,0.0,3736.17,1,0.0,0,RES,NaN,False,Fixed,False,NaN,71909.0,Hot Springs Village,Garland,AR,Entergy,False,Liberty Solar Solutions,False,NaN,NaN,NaN,NaN,NaN,NaN,Sharp,Sharp,Sharp,NU-U230F3,no match,no match,Mono-c-Si,NaN,NaN,False,False,False,0.141090,NaN,NaN,Enphase Energy,NaN,True,False


In [13]:
for col in ['New Construction', 'Tracking', 'Ground Mounted', 
            'Battery System', 'Third-Party Owned', 'Self-Installed', 'BIPV Module #1', 
`           'BIPV Module #2', 'BIPV Module #3', 'Microinverter', 'DC Optimizer']:
    print(dfmini[col].value_counts())

False    67121
True      3019
Name: New Construction, dtype: int64
False    5048
True        8
Name: Tracking, dtype: int64
False    647
True      31
Name: Ground Mounted, dtype: int64
False    72745
True       128
Name: Battery System, dtype: int64
True     49897
False    26101
Name: Third-Party Owned, dtype: int64
False    39840
True       362
Name: Self-Installed, dtype: int64
False    67707
True       566
Name: BIPV Module #1, dtype: int64
False    67707
True       566
Name: BIPV Module #2, dtype: int64
False    67707
True       566
Name: BIPV Module #3, dtype: int64
False    18279
True      3143
Name: Microinverter, dtype: int64
False    21001
True       742
Name: DC Optimizer, dtype: int64


In [15]:
dfmini[['DC Optimizer']].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 100000 entries, 0 to 99999
Data columns (total 1 columns):
DC Optimizer    21743 non-null object
dtypes: object(1)
memory usage: 4.0+ MB


##### Looks like this is working.  Just True, False and NaN

In [16]:
dfmini.to_csv('foob.csv')

In [28]:
dphlegm = pd.read_csv('foob.csv', index_col='row_id', dtype={'DC Optimizer' : np.bool})

C:\Users\saus\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (16,17,18,19,20,23,26,28,42,43,50,51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
# this is my problem.  NaN is a float, so the column has mixed type and dtype doesn't really work then
# because mixed type means object...
type(np.nan)

float

In [34]:
dfmini['Zip Code'].isnull().sum()

107

In [37]:
def fzipToStr(z):
    if pd.isnull(z):
        return np.nan
    else:
        return '{:05d}'.format(int(z))

In [36]:
pd.isnull(np.nan)

True

In [38]:
zipcode = dfmini['Zip Code'].apply(fzipToStr); zipcode.head()

row_id
0    71953
1    72641
2    71801
3    71909
4    71909
Name: Zip Code, dtype: object

In [40]:
zipcode.sort_values().head()

row_id
21    71640
71    71667
75    71730
2     71801
97    71852
Name: Zip Code, dtype: object

In [42]:
zipcode.sort_values().tail(200)

row_id
91816    96067
91926    96067
91813    96067
91925    96067
91824    96067
91917    96067
91829    96067
96798    96073
98642    96073
96590    96080
96911    96080
98387    96080
98455    96087
99372    96087
97135    96088
91901    96094
91922    96094
91974    96094
91978    96094
91939    96094
91900    96094
91919    96094
91858    96094
91995    96094
91905    96094
92006    96094
91849    96094
91843    96094
91863    96094
91996    96094
         ...  
93851      NaN
93924      NaN
94049      NaN
94161      NaN
94325      NaN
94407      NaN
94494      NaN
94498      NaN
94703      NaN
94715      NaN
94761      NaN
94771      NaN
94885      NaN
94906      NaN
95112      NaN
95238      NaN
95330      NaN
95393      NaN
95395      NaN
95478      NaN
95729      NaN
95836      NaN
95890      NaN
95892      NaN
95916      NaN
95986      NaN
95994      NaN
96011      NaN
96063      NaN
96081      NaN
Name: Zip Code, Length: 200, dtype: object

In [29]:
dphlegm.dtypes

index                                             int64
Data Provider                                    object
System ID (from Data Provider)                   object
System ID (Tracking the Sun)                     object
Installation Date                                object
System Size                                     float64
Total Installed Price                           float64
Appraised Value Flag                               bool
Sales Tax Cost                                  float64
Rebate or Grant                                 float64
Performance-Based Incentive (Annual Payment)    float64
Performance-Based Incentives (Duration)           int64
Feed-in Tariff (Annual Payment)                 float64
Feed-in Tariff (Duration)                         int64
Customer Segment                                 object
New Construction                                 object
Tracking                                         object
Tracking Type                                   

In [21]:
dphlegm['DC Optimizer'].value_counts()

False    21001
True       742
Name: DC Optimizer, dtype: int64

In [22]:
thing = dphlegm['DC Optimizer'].astype(np.bool)

In [26]:
thing.dtype

dtype('bool')

In [27]:
type(thing)

pandas.core.series.Series

In [43]:
2.793000e+04

27930.0

In [44]:
5.836735e+00

5.836735

In [3]:
df = pd.DataFrame([[np.nan, 2, np.nan, 0], [3, 4, np.nan, 1],
                  [np.nan, np.nan, np.nan, 5]],
                  columns=list('ABCD'))

In [4]:
df

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1
2,NaN,NaN,NaN,5


In [9]:
df.dropna(axis=0, subset=['B'])

,A,B,C,D
0,NaN,2.0,NaN,0
1,3.0,4.0,NaN,1


This is a sample invocation using and enhancing subplots - might be handy to keep around

Using axes objects is a great approach for this. It helps if you want to interact with multiple figures and sub-plots. To add and manipulate the axes objects directly:
```
fig = pylab.figure(figsize=(12,9))

signal_axes = fig.add_subplot(211)
signal_axes.plot(xs,rawsignal)

fft_axes = fig.add_subplot(212)
fft_axes.set_title("FFT")
fft_axes.set_autoscaley_on(False)
fft_axes.set_ylim([0,1000])
fft = scipy.fft(rawsignal)
fft_axes.plot(abs(fft))
```
